In [2]:
import h5py
from glob import glob
import sys, scipy
from scipy.stats import chi2
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.utils import shuffle
from prettytable import PrettyTable

import gpustat
gpustat.print_gpustat()
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # Some versions of HDF5 require this
os.environ['CUDA_VISIBLE_DEVICES']='2' # This is to choose which GPU to use

import h5py
import numpy as np

from torch.utils.data import Dataset, DataLoader # 
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F

gpu-4-culture-plate-sm  Tue Jul 16 15:48:12 2019
[0] GeForce GTX 1080 | 23'C,   0 % |     0 /  8114 MB |
[1] GeForce GTX 1080 | 25'C,   0 % |     0 /  8114 MB |
[2] GeForce GTX 1080 | 22'C,   0 % |     0 /  8114 MB |
[3] GeForce GTX 1080 | 22'C,   0 % |     0 /  8114 MB |
[4] GeForce GTX 1080 | 23'C,   0 % |     0 /  8114 MB |
[5] GeForce GTX 1080 | 28'C,   0 % |     0 /  8114 MB |
[6] GeForce GTX 1080 | 25'C,   0 % |  3817 /  8114 MB | thong(3807M)
[7] GeForce GTX 1080 | 24'C,   0 % |     0 /  8114 MB |


In [3]:
INPUT_LENGTH = 20 # number of particles to consider in 1 event
INPUT_FEATURE = 4 # number of features. use pt, eta, phi, and pid
BATCH_SIZE = 30
PT_SCALE = 10
MAX_EPOCH = 200
# Maurizio's Delphes data
# this is the location on the caltech machine. I copied the file to /eos/project/d/dshep/Thong_tmp/ on lxplus
base_dir = '/bigdata/shared/L1AnomalyDetection/qcd_lepFilter_13TeV/' 
base_file = '/bigdata/shared/L1AnomalyDetection/qcd_all.npy'
bsm_dir = '/bigdata/shared/L1AnomalyDetection/Ato4l_lepFilter_13TeV/'
bsm_file = '/bigdata/shared/L1AnomalyDetection/Ato4l.npy'
model_file = 'DelphesModel.torch'

# particle is ordered: MET + 10 e + 10 mu + 20 jet
# We will take 4 ele + 4 muon + 12 jets = 20 objects in total

PARTICLE_TO_USE = np.asarray([False]*41)
PARTICLE_TO_USE[1:5] = True # 4 ele
PARTICLE_TO_USE[11:15] = True # 4 muon
PARTICLE_TO_USE[21:33] = True # 12 jets
print(sum(PARTICLE_TO_USE))
FLAT_FEATURES = INPUT_LENGTH*INPUT_FEATURE

20


In [5]:
class SimpleEventSequence(Dataset):
    def __init__(self, data_x):
        self.len = data_x.shape[0]
        self.data_x = torch.from_numpy(data_x).float()
        #self.data_y = torch.from_numpy(data_y)
        
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.data_x[idx].contiguous().view(FLAT_FEATURES)


In [6]:
class Autoencoder(nn.Module):
    def __init__(self, n_features):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_features, int(n_features*2/3)),
            nn.ReLU(True),
            nn.Linear(int(n_features*2/3), int(n_features/3)),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(int(n_features/3), int(n_features*2/3)),
            nn.ReLU(True),
            nn.Linear(int(n_features*2/3), n_features)
            )

    def forward(self, x):
        #print(x.shape)
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [7]:
# Loading data
all_qcd = np.load(base_file).astype(np.float32)[:,PARTICLE_TO_USE,:]
size_qcd = len(all_qcd)
qcd_train = all_qcd[:int(size_qcd/2)]
qcd_val = all_qcd[int(size_qcd/2):]
print(all_qcd.shape)

x_mean = np.mean(qcd_train.reshape((-1,4)), axis=0)
x_std = np.std(qcd_train.reshape((-1,4)), axis=0)
print(x_mean)
print(x_std)

qcd_train_norm = qcd_train
qcd_val_norm = qcd_val

# Normalize pt, eta, phi
for i in range(3): 
    qcd_train_norm[:,:,i] = (qcd_train[:,:,i] - x_mean[i])/(x_std[i])
    qcd_val_norm[:,:,i] = (qcd_val[:,:,i] - x_mean[i])/(x_std[i])
    

train_loader = DataLoader(SimpleEventSequence(qcd_train_norm), 
                                    batch_size = BATCH_SIZE, shuffle=False,num_workers=3)
val_loader = DataLoader(SimpleEventSequence(qcd_val_norm), 
                                    batch_size = BATCH_SIZE*10, shuffle=False,num_workers=3)



(13780026, 20, 4)
[ 7.1738086e+00 -2.1654021e-05 -7.8462181e-05  3.8649943e-01]
[20.075266    0.6121493   0.6666415   0.87105507]


In [6]:

model = Autoencoder(FLAT_FEATURES).cuda()
print(model)
trainablePars = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)


Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=80, out_features=53, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=53, out_features=26, bias=True)
    (3): ReLU(inplace)
  )
  (decoder): Sequential(
    (0): Linear(in_features=26, out_features=53, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=53, out_features=80, bias=True)
  )
)

Trainable parameters: 11448


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.L1Loss()

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 
                              mode='min',
                              factor=0.3,
                              patience=3,
                              verbose=1,
                              threshold=1e-4,
                              cooldown=2,
                              min_lr=1e-7
                             )

train_loss = []
loss_history = {'train': [], 'val': []}

min_loss, stale_epochs = 1e6, 0
for epoch in range(MAX_EPOCH):
    
    batch_loss = []
    # Training
    model.train
    for batch_idx, local_x in enumerate(train_loader):
        local_x = Variable(local_x).float().cuda() 
        # I use an old version of Pytorch. For version > 0.4, feel free to remove Variable wrapper
        x_prime = model(local_x)
        loss = criterion(x_prime, local_x)
        batch_loss.append(loss.data[0])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    loss_history['train'].append(sum(batch_loss)/len(train_loader))

    # Evaluation
    batch_loss = []
    model.eval()
    
    for batch_idx, local_x in enumerate(val_loader):
        local_x = Variable(local_x, volatile=True).float().cuda() 
        # I use an old version of Pytorch. For version > 0.4, feel free to remove Variable wrapper
        x_prime = model(local_x)
        loss = criterion(x_prime, local_x)
        batch_loss.append(loss.data[0])
    
    this_loss = sum(batch_loss)/len(val_loader)
    loss_history['val'].append(this_loss)
    
    print('Epoch [{}/{}], Train loss: {:.4f}, Validation loss: {:.4f}'.format(epoch + 1, 
                                                                       MAX_EPOCH, 
                                                                       loss_history['train'][-1],
                                                                       this_loss))
    
    if stale_epochs > 10:
        print("Early stopped")
        break
        
    if this_loss < min_loss:
        min_loss = this_loss
        stale_epochs = 0
        torch.save(model.state_dict(), '_best_'+model_file)
        print("New best loss. Model saved in _best_{}".format(model_file))
    else:
        stale_epochs += 1
        print("Stale = {}. Best loss = {}".format(stale_epochs, min_loss))
    
    

Epoch [1/200], Train loss: 0.0641, Validation loss: 0.0321
New best loss. Model saved in _best_DelphesModel.torch
Epoch [2/200], Train loss: 0.0304, Validation loss: 0.0309
New best loss. Model saved in _best_DelphesModel.torch
Epoch [3/200], Train loss: 0.0299, Validation loss: 0.0307
New best loss. Model saved in _best_DelphesModel.torch
Epoch [4/200], Train loss: 0.0297, Validation loss: 0.0301
New best loss. Model saved in _best_DelphesModel.torch
Epoch [5/200], Train loss: 0.0296, Validation loss: 0.0307
Stale = 1. Best loss = 0.03008725909999496
Epoch [6/200], Train loss: 0.0295, Validation loss: 0.0298
New best loss. Model saved in _best_DelphesModel.torch
Epoch [7/200], Train loss: 0.0295, Validation loss: 0.0303
Stale = 1. Best loss = 0.029823871334373923
Epoch [8/200], Train loss: 0.0295, Validation loss: 0.0302
Stale = 2. Best loss = 0.029823871334373923
Epoch [9/200], Train loss: 0.0295, Validation loss: 0.0300
Stale = 3. Best loss = 0.029823871334373923
Epoch [10/200], Tra

# Evaluation

In [8]:
model = Autoencoder(FLAT_FEATURES).cuda()
model.load_state_dict(torch.load('_best_{}'.format(model_file)))

model.eval()
volatile=True

criterion = nn.L1Loss(reduce=False)
sm_loss = np.array([])
for batch_idx, local_x in enumerate(val_loader):
    local_x = Variable(local_x, volatile=True).float().cuda() 
    # I use an old version of Pytorch. For version > 0.4, feel free to remove Variable wrapper
    x_prime = model(local_x)
    loss = torch.sum(criterion(x_prime, local_x), dim=1).view(-1,1).data.cpu().numpy()
    sm_loss = np.concatenate((sm_loss, loss)) if sm_loss.size else loss
    